## Perplexity & Burstiness

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import torch

# 加载预训练模型
def load_model(model_name="gpt2"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.eval()
    return tokenizer, model

# 计算困惑度
def compute_perplexity(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs.input_ids)
    loss = outputs.loss.item()
    return np.exp(loss)

# 计算突发性
def compute_burstiness(sentences):
    sentence_lengths = [len(sentence.split()) for sentence in sentences]
    mean_length = np.mean(sentence_lengths)
    std_length = np.std(sentence_lengths)
    return std_length / mean_length  # 标准差与均值比

# GPTZero 检测
def gptzero_detect(text, tokenizer, model):
    sentences = text.split(". ")  # 简单句子分割
    perplexities = [compute_perplexity(sentence, tokenizer, model) for sentence in sentences]
    avg_perplexity = np.mean(perplexities)
    burstiness = compute_burstiness(sentences)

    # 简单阈值判断
    is_generated = avg_perplexity < 30 and burstiness < 0.5
    return {"avg_perplexity": avg_perplexity, "burstiness": burstiness, "is_generated": is_generated}

# 示例运行
if __name__ == "__main__":
    tokenizer, model = load_model("gpt2")
    text = "This is a GPT-generated sentence. It is very uniform and lacks diversity."
    result = gptzero_detect(text, tokenizer, model)
    print(f"Detection Result: {result}")


## Curvature of log probability function

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 加载预训练语言模型
def load_model(model_name="gpt2"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.eval()
    return tokenizer, model

# 计算对数概率曲率
def compute_log_probability_curvature(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"]
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        logits = outputs.logits
        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
        probabilities = torch.exp(log_probs)
        curvature = probabilities.var().item()  # 用方差近似曲率
    return curvature

# 生成扰动文本
def perturb_text(text, num_perturbations=10):
    words = text.split()
    perturbed_texts = []
    for _ in range(num_perturbations):
        if len(words) > 1:
            idx = torch.randint(0, len(words), (1,)).item()
            words[idx] = "[MASK]"  # 简单替换模拟扰动
        perturbed_texts.append(" ".join(words))
    return perturbed_texts

# DetectGPT 检测
def detect_gpt(text, tokenizer, model, num_perturbations=10, threshold=0.05):
    original_curvature = compute_log_probability_curvature(text, tokenizer, model)
    perturbed_texts = perturb_text(text, num_perturbations)
    perturbed_curvatures = [compute_log_probability_curvature(t, tokenizer, model) for t in perturbed_texts]
    
    # 计算曲率变化
    avg_perturbed_curvature = sum(perturbed_curvatures) / len(perturbed_curvatures)
    delta_curvature = abs(avg_perturbed_curvature - original_curvature)
    
    # 根据阈值判断是否生成
    is_generated = delta_curvature < threshold
    return {
        "original_curvature": original_curvature,
        "avg_perturbed_curvature": avg_perturbed_curvature,
        "delta_curvature": delta_curvature,
        "is_generated": is_generated
    }

# 示例运行
if __name__ == "__main__":
    tokenizer, model = load_model("gpt2")
    text = "This is a GPT-generated sentence."
    result = detect_gpt(text, tokenizer, model)
    print(f"Detection Result: {result}")
